In [1]:
import warnings
warnings.filterwarnings('ignore')

In [32]:
#import useful libraries
import pandas as pd
import numpy as np
from tqdm import tqdm
from random import randint
from datetime import datetime
from functions import *

In [3]:
data = pd.read_csv('bank_transactions.csv', delimiter=',')

In [4]:
data.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5


# Finding Similar Customers

## 1.1 Set up the date

clean the dataset

In [4]:
#check for any NA values
any(data.isna())
data.dropna(inplace=True)

convert string to datetime

In [5]:
data.CustomerDOB = pd.to_datetime(data.CustomerDOB)
data.TransactionDate = pd.to_datetime(data.TransactionDate)

data.drop(data[data.CustomerDOB.dt.year > 1998].index, axis = 0, inplace = True)

data.drop(data[data.CustomerDOB == 1800].index, axis = 0, inplace=True)

data = data.reset_index()

We are going to keep only the rows We are going to use later in the query. Thus, we are going to get rid of *TransactionID* and *CustomerID*

In [6]:
data = data[['CustomerDOB','CustGender','CustLocation','CustAccountBalance','TransactionDate','TransactionTime','TransactionAmount (INR)']]

In [10]:
def DOB_to_age(df):
    now = int(datetime.strftime(datetime.today().date()).split('-')[0])
    age = df.apply(lambda x: now - x)
    return age 

In [7]:
data['CustomerDOB'] = data['CustomerDOB'].apply(lambda x: int(datetime.strftime(x, "%Y-%m-%w").split('-')[0]))
today = int(datetime.strftime(datetime.today().date(), "%Y-%m-%w").split('-')[0])
data['Age'] = data['CustomerDOB'].apply(lambda x: today - x)

In [9]:
import time
week_day = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
data['TransactionDay'] = data['TransactionTime'].apply(time.ctime)
data['Transaction_Hour'] = data['TransactionDay'].apply(lambda x : x[10:-4])
data['Transaction_weekday']     = data['TransactionDate'].apply(lambda x : week_day[pd.Timestamp(x).dayofweek])

In [ ]:
def day_of_week(val):
  if val == "Mon":
    return 1
  elif val == "Tue":
    return 2
  elif val == "Wed":
    return 3
  elif val == "Thu":
    return 4
  elif val == "Fri":
    return 5
  elif val == "Sat":
    return 6
  elif val == "Sun":
    return 7

In [10]:
data['Transaction_weekday'] = data['Transaction_weekday'].apply(lambda x: day_of_week(x))

In [ ]:
def time_ranges(val):
  tmp = int(val.split(':')[0])

  if tmp >= 0 and tmp < 2:
    return 1
  elif tmp >= 2 and tmp < 4:
    return 2
  elif tmp >= 4 and tmp < 6:
    return 3
  elif tmp >= 6 and tmp < 8:
    return 4
  elif tmp >= 8 and tmp < 10:
    return 5
  elif tmp >= 10 and tmp < 12:
    return 6
  elif tmp >= 12 and tmp < 14:
    return 7
  elif tmp >= 14 and tmp < 16:
    return 8
  elif tmp >= 16 and tmp < 18:
    return 9
  elif tmp >= 18 and tmp < 20:
    return 10
  elif tmp >= 20 and tmp < 22:
    return 11
  else: return 12

In [11]:
data['Time_range'] = data['Transaction_Hour'].apply(lambda x: time_ranges(x))

In [12]:
data

,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),Age,TransactionDay,Transaction_Hour,Transaction_weekday,Time_range
0,1994,F,JAMSHEDPUR,17819.05,2016-02-08,143207,25.0,28,Fri Jan 2 16:46:47 1970,16:46:47,1,9
1,1996,F,MUMBAI,17874.44,2016-02-08,142712,459.0,26,Fri Jan 2 16:38:32 1970,16:38:32,1,9
2,1973,F,MUMBAI,866503.21,2016-02-08,142714,2060.0,49,Fri Jan 2 16:38:34 1970,16:38:34,1,9
3,1988,F,NAVI MUMBAI,6714.43,2016-02-08,181156,1762.5,34,Sat Jan 3 03:19:16 1970,03:19:16,1,2
4,1972,F,ITANAGAR,53609.20,2016-02-08,173940,676.0,50,Sat Jan 3 01:19:00 1970,01:19:00,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
965271,1990,M,NEW DELHI,7635.19,2016-09-18,184824,799.0,32,Sat Jan 3 04:20:24 1970,04:20:24,7,3
965272,1992,M,NASHIK,27311.42,2016-09-18,183734,460.0,30,Sat Jan 3 04:02:14 1970,04:02:14,7,3
965273,1989,M,HYDERABAD,221757.06,2016-09-18,183313,770.0,33,Sat Jan 3 03:55:13 1970,03:55:13,7,2
965274,1978,M,VISAKHAPATNAM,10117.87,2016-09-18,184706,1000.0,44,Sat Jan 3 04:18:26 1970,04:18:26,7,3


In [13]:
#after the feature engineering let's drop some columns
data = data.drop(['CustomerDOB','TransactionDate', 'TransactionTime', 'TransactionDay', 'Transaction_Hour'], axis = 1)

In [14]:
data

,CustGender,CustLocation,CustAccountBalance,TransactionAmount (INR),Age,Transaction_weekday,Time_range
0,F,JAMSHEDPUR,17819.05,25.0,28,1,9
1,F,MUMBAI,17874.44,459.0,26,1,9
2,F,MUMBAI,866503.21,2060.0,49,1,9
3,F,NAVI MUMBAI,6714.43,1762.5,34,1,2
4,F,ITANAGAR,53609.20,676.0,50,1,1
...,...,...,...,...,...,...,...
965271,M,NEW DELHI,7635.19,799.0,32,7,3
965272,M,NASHIK,27311.42,460.0,30,7,3
965273,M,HYDERABAD,221757.06,770.0,33,7,2
965274,M,VISAKHAPATNAM,10117.87,1000.0,44,7,3


First thing to do is convert our values in int32 dtype in order to apply the hash function

In [15]:
data.CustGender = data['CustGender'].apply(string_to_int)
data.CustLocation = data['CustLocation'].apply(string_to_int)
data.CustAccountBalance = data['CustAccountBalance'].apply(float_to_int)
data['TransactionAmount (INR)'] = data['TransactionAmount (INR)'].apply(float_to_int)


In [16]:
data

,CustGender,CustLocation,CustAccountBalance,TransactionAmount (INR),Age,Transaction_weekday,Time_range
0,70,755,17819,25,28,1,9
1,70,443,17874,459,26,1,9
2,70,443,866503,2060,49,1,9
3,70,777,6714,1762,34,1,2
4,70,583,53609,676,50,1,1
...,...,...,...,...,...,...,...
965271,77,624,7635,799,32,7,3
965272,77,446,27311,460,30,7,3
965273,77,644,221757,770,33,7,2
965274,77,968,10117,1000,44,7,3


In [17]:
#obtain max value for x
data.max()

CustGender                       84
CustLocation                   2609
CustAccountBalance         55369688
TransactionAmount (INR)     1560034
Age                             222
Transaction_weekday               7
Time_range                       12
dtype: int64

In [18]:
#let's get the next prime number bigger than x

from sympy import nextprime

max_value_of_x = nextprime(55369688)

## 1.2 Fingerprint Hashing

The hash function we want to implement is in the following form: <br>
$\begin{equation} h(x)=(ax+b)\mod{c}\end{equation}$ <br>
where: <br>
- *x*: input value
- *a*, *b*: randomly choosen integers less than the maximum value of *x*
- *c*: the next prime number bigger than the maximum value of *x*

Now it's time to implement the MinHash

In [40]:
def hash(val, a, b, c):
    return (a*val+b)%c
    
def min_hash(info,c ,list_of_tuples):

    signatures = []

    for idx, x in enumerate(info):
        hash_value = []
        for values in list_of_tuples:
            a,b = values
            ans = hash(x,a,b,c)
            hash_value.append(ans)
        signatures.append(int(min(hash_value)))
        
    return signatures

    
def random_coef(n_hash, M):

    a = []
    b = []

    for _ in range(n_hash):
        a_i = randint(0,M)
        a.append(a_i)
        b_i = randint(0,M)
        b.append(b_i)
    return a,b

In [19]:
c = max_value_of_x
M = 2**32 -1                           
n_hash = 10

a,b = random_coef(n_hash, M)

list_of_tuples = list(
    map(
        lambda x, y: (x, y),
        a,
        b
    )
)
data['min-hash']  = [min_hash([*x[1]],c,list_of_tuples) for x in (data.iterrows())]

In [18]:
data

,CustGender,CustLocation,CustAccountBalance,TransactionAmount (INR),Age,Transaction_weekday,Time_range,min-hash
0,70,755,17819,25,28,1,9,"[1102418, 10138512, 6710221, 6367078, 8394840,..."
1,70,443,17874,459,26,1,9,"[1102418, 5221397, 10298307, 10882374, 1391475..."
2,70,443,866503,2060,49,1,9,"[1102418, 5221397, 15640709, 9991438, 4938426,..."
3,70,777,6714,1762,34,1,2,"[1102418, 10178519, 12196082, 1673920, 2303987..."
4,70,583,53609,676,50,1,1,"[1102418, 6809195, 12713132, 1492298, 10401131..."
...,...,...,...,...,...,...,...,...
965271,77,624,7635,799,32,7,3,"[11258163, 280520, 4697713, 6078549, 3831087, ..."
965272,77,446,27311,460,30,7,3,"[11258163, 2274904, 17351856, 1106841, 5225369..."
965273,77,644,221757,770,33,7,2,"[11258163, 4851909, 16994735, 149215, 6039330,..."
965274,77,968,10117,1000,44,7,3,"[11258163, 79422, 11946851, 1177931, 2863661, ..."


## 1.3 Locality Sensitive Hashing (LSH)

Now let's apply the same idea on the query, and then apply the Jaccard similarity since MinHash is stricly related to it

In [20]:
query = pd.read_csv('query_users.csv')

In [58]:
query.head()

,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,27/7/78,M,DELHI,94695.61,2/9/16,140310,65.0
1,6/11/92,M,PANCHKULA,7584.09,2/9/16,120214,6025.0
2,14/8/91,M,PATNA,7180.60,10/8/16,221732,541.5
3,3/1/87,M,CHENNAI,56847.75,29/8/16,144138,1000.0
4,4/1/95,M,GURGAON,84950.13,25/9/16,233309,80.0


pre-processing of the query

In [21]:
#convert
query.CustomerDOB = pd.to_datetime(query.CustomerDOB)
query.TransactionDate = pd.to_datetime(query.TransactionDate)

In [22]:
query['CustomerDOB'] = query['CustomerDOB'].apply(lambda x: int(datetime.strftime(x, "%Y-%m-%w").split('-')[0]))
today = int(datetime.strftime(datetime.today().date(), "%Y-%m-%w").split('-')[0])
query['Age'] = query['CustomerDOB'].apply(lambda x: today - x)

In [23]:
week_day = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
query['TransactionDay'] = query['TransactionTime'].apply(time.ctime)
query['Transaction_Hour'] = query['TransactionDay'].apply(lambda x : x[10:-4])
query['Transaction_weekday'] = query['TransactionDate'].apply(lambda x : week_day[pd.Timestamp(x).dayofweek])

In [24]:
query.head()

,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),Age,TransactionDay,Transaction_Hour,Transaction_weekday
0,1978,M,DELHI,94695.61,2016-02-09,140310,65.0,44,Fri Jan 2 15:58:30 1970,15:58:30,Tue
1,1992,M,PANCHKULA,7584.09,2016-02-09,120214,6025.0,30,Fri Jan 2 10:23:34 1970,10:23:34,Tue
2,1991,M,PATNA,7180.60,2016-10-08,221732,541.5,31,Sat Jan 3 14:35:32 1970,14:35:32,Sat
3,1987,M,CHENNAI,56847.75,2016-08-29,144138,1000.0,35,Fri Jan 2 17:02:18 1970,17:02:18,Mon
4,1995,M,GURGAON,84950.13,2016-09-25,233309,80.0,27,Sat Jan 3 17:48:29 1970,17:48:29,Sun


In [25]:
query['Transaction_weekday'] = query['Transaction_weekday'].apply(lambda x: day_of_week(x))
query['Time_range'] = query['Transaction_Hour'].apply(lambda x: time_ranges(x))
query.CustGender = query['CustGender'].apply(string_to_int)
query.CustLocation = query['CustLocation'].apply(string_to_int)
query.CustAccountBalance = query['CustAccountBalance'].apply(float_to_int)
query['TransactionAmount (INR)'] = query['TransactionAmount (INR)'].apply(float_to_int)

In [26]:
query = query.drop(['CustomerDOB','TransactionDate', 'TransactionTime', 'TransactionDay', 'Transaction_Hour'], axis = 1)

In [27]:
query.head()

,CustGender,CustLocation,CustAccountBalance,TransactionAmount (INR),Age,Transaction_weekday,Time_range
0,77,358,94695,65,44,2,8
1,77,663,7584,6025,30,2,6
2,77,372,7180,541,31,6,8
3,77,502,56847,1000,35,1,9
4,77,531,84950,80,27,7,9


In [28]:
query['min-hash']  = [min_hash([*x[1]],c,list_of_tuples) for x in (query.iterrows())]

In [29]:
query.head()

,CustGender,CustLocation,CustAccountBalance,TransactionAmount (INR),Age,Transaction_weekday,Time_range,min-hash
0,77,358,94695,65,44,2,8,"[1515530, 10025985, 8679111, 4409915, 10584350..."
1,77,663,7584,6025,30,2,6,"[1515530, 11253511, 5732987, 3160832, 2367083,..."
2,77,372,7180,541,31,6,8,"[1515530, 3096208, 768250, 4039217, 1083672, 5..."
3,77,502,56847,1000,35,1,9,"[1515530, 304665, 16141755, 14998627, 2585127,..."
4,77,531,84950,80,27,7,9,"[1515530, 6208550, 687, 5322309, 6487507, 1126..."


Now let's compute Jaccard's similarity. <br>
Jaccard's similarity is defined as:
$J(A,B)=\dfrac{|A \cap B|}{|A \cup B|}$

In [39]:
def jaccard(x,y):
    A = set([i for i in x])
    B = set([j for j in y])

    return len(A.intersection(B))/len(A.union(B))

Let's now compute the Jaccard Similarity between the query and the original dataframe on the minhash

In [175]:
resultDF = pd.DataFrame(columns=['Query Index', 'Data Index-Jaccard Similarity'])

In [188]:
import time

In [189]:
#Test with a threshold set to 0.6
result = {}

start = time.time()
threshold = 0.6
for idx_1, val_1 in enumerate(query['min-hash']):
    jaccard_list = []
    index_list = []
    for idx_2, val_2 in enumerate(data['min-hash']):
        if jaccard(val_1, val_2) > threshold:
            index_list.append(idx_2)
            jaccard_list.append(jaccard(val_1, val_2))
            result.update({idx_1: sorted(list(zip(index_list,jaccard_list)), key = lambda x: x[1], reverse=True)})
end = time.time() - start

In [191]:
print(end)

86.84473609924316


In [180]:
resultDF['Query Index'] = list(result.keys())
resultDF['Data Index-Jaccard Similarity'] = list(result.values())

In [179]:
resultDF

,Query Index,Data Index-Jaccard Similarity
0,0,"[(687184, 1.0), (76578, 0.75)]"
1,1,"[(670151, 1.0), (419461, 0.75), (946865, 0.75)]"
2,2,"[(290921, 1.0)]"
3,3,"[(588909, 1.0), (736637, 0.625)]"
4,4,"[(10697, 1.0), (429598, 0.625), (837934, 0.625)]"
5,5,"[(862651, 1.0), (869679, 0.75)]"
6,6,"[(569281, 1.0), (139122, 0.75), (139345, 0.75)..."
7,7,"[(73174, 1.0), (210548, 0.75), (329966, 0.75),..."
8,8,"[(551951, 1.0), (561493, 0.75)]"
9,10,"[(116689, 1.0), (480784, 1.0), (193334, 0.75),..."


In [194]:
#Test with a threshold set to 0.7
result = {}
threshold = 0.7

start = time.time()
for idx_1, val_1 in enumerate(query['min-hash']):
    jaccard_list = []
    index_list = []
    for idx_2, val_2 in enumerate(data['min-hash']):
        if jaccard(val_1, val_2) > threshold:
            index_list.append(idx_2)
            jaccard_list.append(jaccard(val_1, val_2))
            result.update({idx_1: sorted(list(zip(index_list,jaccard_list)), key = lambda x: x[1], reverse=True)})
end = time.time() - start

In [195]:
end

87.40046095848083

In [184]:
len(result)

35

In [185]:
#Test with a threshold set to 0.8
result = {}

threshold = 0.8

start = time.time()
for idx_1, val_1 in enumerate(query['min-hash']):
    jaccard_list = []
    index_list = []
    for idx_2, val_2 in enumerate(data['min-hash']):
        if jaccard(val_1, val_2) > threshold:
            index_list.append(idx_2)
            jaccard_list.append(jaccard(val_1, val_2))
            result.update({idx_1: list(zip(index_list,jaccard_list))})

end = time.time() - start

In [193]:
end

86.70222210884094

In [187]:
len(result)

31